In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import inspect
import sys
pd.set_option('display.max_colwidth', None)

In [3]:
import sys
sys.path.insert(0,'/home/rahmah/code/marvinliebisch/forecasting-electricity-prices')

In [7]:
from energy_price_predictions.ml_logic.data_import import *
from energy_price_predictions.ml_logic.preprocessing import *
from energy_price_predictions.ml_logic.registry import *
from energy_price_predictions.ml_logic.model import *

In [8]:
def preprocess():
    df = import_merged_data()
    df = df.dropna()
    X, y = normalization(df)
    X=pd.DataFrame(X)
    y=pd.DataFrame(y)
    y_shape = y.shape[1]

    def sequence_data(X, y,
                    n_observation_X, n_observation_y,
                    n_sequence_train,n_sequence_val,n_sequence_test,
                    val_cutoff,test_cutoff):


        sample_list_train = list(range(0, int(len(X)*val_cutoff-n_observation_y-n_observation_X)))
        sample_list_val = list(range(int(len(X)*val_cutoff),int(len(X)*test_cutoff)))
        sample_list_test= list(range(int(len(X)*test_cutoff),int(len(X)-n_observation_y-n_observation_X)))

        random.shuffle(sample_list_train)
        random.shuffle(sample_list_val)
        random.shuffle(sample_list_test)

        X_train=np.zeros((n_sequence_train, n_observation_X, X.shape[1]))
        X_val=np.zeros((n_sequence_val, n_observation_X, X.shape[1]))
        X_test=np.zeros((n_sequence_test, n_observation_X, X.shape[1]))

        y_train=np.zeros((n_sequence_train, n_observation_y, 1))
        y_val=np.zeros((n_sequence_val, n_observation_y, 1))
        y_test=np.zeros((n_sequence_test, n_observation_y, 1))


        def create_sequence(X_,y_,sample_list,n_sequence):
            index=0
            for i in sample_list[0:n_sequence]:
                X_[index] = X.iloc[i:i + n_observation_X].values
                y_[index]= y.iloc[i + n_observation_X:i + n_observation_X + n_observation_y].values
                index=index+1
            return X_, y_

        X_train, y_train = create_sequence(X_train,y_train,sample_list_train,n_sequence_train)
        X_val, y_val = create_sequence(X_val,y_val,sample_list_val,n_sequence_val)
        X_test, y_test = create_sequence(X_test,y_test,sample_list_test,n_sequence_test)

        return X_train, X_val, X_test, y_train, y_val,y_test


    n_observation_X=24 * 7*4  # For example, a week of data for the sequence
    n_observation_y=24 # We would like to forecast the 24 prices of the next day during the auction of today
    n_sequence_train=60
    n_sequence_val=5
    n_sequence_test=5
    val_cutoff=0.8
    test_cutoff=0.9


    X_train, X_val, X_test, y_train, y_val,y_test = sequence_data(X, y,
                    n_observation_X, n_observation_y,
                    n_sequence_train,n_sequence_val,n_sequence_test,
                    val_cutoff,test_cutoff)
    return X_train, X_val, X_test, y_train, y_val,y_test, y_shape

In [9]:
@mlflow_run
def train(X_train, y_train, X_val, y_val, y_shape) -> float:
    """
    - Train on the preprocessed dataset (which should be ordered by date)
    - Store training results and model weights

    Return val_mae as float
    """

    # Train model using `model.py`
    # model = load_model()
    # if model is None:
    model = initialize_model(input_shape=X_train.shape[1:], output_shape=y_shape)
    model = compile_model(model)
    model, history = train_model(model, X_train, y_train,
                                 validation_data=(X_val, y_val))

    val_mae = np.min(history.history['val_mae'])

    params = dict(
        context="train",
        training_set_size=len(X_train),
    )

    save_results(params=params, metrics=dict(mae=val_mae))
    save_model(model=model)

    # The latest model should be moved to staging
    mlflow_transition_model(current_stage="None", new_stage="Staging")

    return val_mae

In [10]:
@mlflow_run
def evaluate(X_new, y_new, stage: str = "Production") -> float:
    """
    Evaluate the performance of the latest production model on processed data
    Return mae as float
    """

    model = load_model(stage=stage)
    assert model is not None

    metrics_dict = evaluate_model(model=model, X=X_new, y=y_new)
    mae = metrics_dict["mae"]

    params = dict(
        context="evaluate", # Package behavior
    )

    save_results(params=params, metrics=metrics_dict)

    return mae


In [11]:
X_train, X_val, X_test, y_train, y_val,y_test, y_shape = preprocess()

Importing energy_dataset data from /home/rahmah/code/marvinliebisch/forecasting-electricity-prices/raw_data/energy_dataset.csv...
Importing weather_features data from /home/rahmah/code/marvinliebisch/forecasting-electricity-prices/raw_data/weather_features.csv...


In [12]:
train(X_train, y_train, X_val, y_val, y_shape)

Epoch 1/1000
4/4 [==============================] - 14s 2s/step - loss: 2240.2000 - mae: 45.0564 - val_loss: 2527.9690 - val_mae: 49.7263
Epoch 2/1000
4/4 [==============================] - 6s 2s/step - loss: 1676.1918 - mae: 38.5036 - val_loss: 2184.3054 - val_mae: 46.1416
Epoch 3/1000
4/4 [==============================] - 7s 2s/step - loss: 1460.7188 - mae: 35.6804 - val_loss: 1962.0199 - val_mae: 43.6665
Epoch 4/1000
4/4 [==============================] - 7s 2s/step - loss: 1291.0057 - mae: 33.2873 - val_loss: 1753.9880 - val_mae: 41.2156
Epoch 5/1000
4/4 [==============================] - 7s 2s/step - loss: 1138.4594 - mae: 31.0020 - val_loss: 1567.6096 - val_mae: 38.8890
Epoch 6/1000
4/4 [==============================] - 7s 2s/step - loss: 1001.8949 - mae: 28.8248 - val_loss: 1390.9602 - val_mae: 36.5472
Epoch 7/1000
4/4 [==============================] - 8s 2s/step - loss: 876.3168 - mae: 26.6981 - val_loss: 1231.4720 - val_mae: 34.2960
Epoch 8/1000
4/4 [=======================

4/4 [==============================] - 6s 2s/step - loss: 204.9236 - mae: 10.6348 - val_loss: 157.0894 - val_mae: 11.6542
Epoch 62/1000
4/4 [==============================] - 6s 2s/step - loss: 204.9244 - mae: 10.6348 - val_loss: 157.0978 - val_mae: 11.6545
Epoch 63/1000
4/4 [==============================] - 6s 2s/step - loss: 204.9251 - mae: 10.6348 - val_loss: 157.1060 - val_mae: 11.6549
Epoch 64/1000
4/4 [==============================] - 6s 2s/step - loss: 204.9259 - mae: 10.6348 - val_loss: 157.1137 - val_mae: 11.6552
Epoch 65/1000
4/4 [==============================] - 6s 2s/step - loss: 204.9266 - mae: 10.6347 - val_loss: 157.1214 - val_mae: 11.6555
Epoch 66/1000
4/4 [==============================] - 6s 2s/step - loss: 204.9273 - mae: 10.6347 - val_loss: 157.1288 - val_mae: 11.6558
Epoch 67/1000
1/1 [==============================] - 1s 1s/step


2023-03-28 09:52:28.558 WARNING absl: Found untraced functions such as lstm_cell_3_layer_call_fn, lstm_cell_3_layer_call_and_return_conditional_losses, lstm_cell_4_layer_call_fn, lstm_cell_4_layer_call_and_return_conditional_losses, lstm_cell_5_layer_call_fn while saving (showing 5 of 6). These functions will not be directly callable after loading.
2023-03-28 09:52:30.248 INFO    tensorflow: Assets written to: /tmp/tmpko1t15mx/model/data/model/assets
2023/03/28 09:52:45 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/rahmah/.pyenv/versions/3.10.6/envs/forecasting-electricity-prices/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
2023/03/28 09:52:49 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inp

11.651476860046387